In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [17]:
##공분산행렬 전처리
covmat = pd.read_excel('자산배분 실습.xlsx',
                       header = 4,index_col= 1)
covmat.drop('Unnamed: 0',axis=1,inplace=True)
covmat = covmat.iloc[:10].copy()
covmat

,US Large,Europe,Japan,Emerging,US Long Term,US Mid Term,US REITs,International REITs,Gold,Commodities
US Large,0.023252,0.019982,0.017300,0.022822,-0.004060,-0.002021,0.019181,0.012286,-0.000357,0.005273
Europe,0.019982,0.031276,0.019930,0.028016,-0.005252,-0.002429,0.020870,0.017219,0.002426,0.008212
Japan,0.017300,0.019930,0.037028,0.024458,-0.003813,-0.001881,0.015824,0.013942,0.002175,0.005775
Emerging,0.022822,0.028016,0.024458,0.046069,-0.005900,-0.002948,0.026462,0.021058,0.003875,0.010617
US Long Term,-0.004060,-0.005252,-0.003813,-0.005900,0.010108,0.004733,-0.003653,-0.003573,0.001124,-0.002675
US Mid Term,-0.002021,-0.002429,-0.001881,-0.002948,0.004733,0.002773,-0.001793,-0.001629,0.000787,-0.001119
US REITs,0.019181,0.020870,0.015824,0.026462,-0.003653,-0.001793,0.038965,0.016408,0.000774,0.005875
International REITs,0.012286,0.017219,0.013942,0.021058,-0.003573,-0.001629,0.016408,0.026134,0.002801,0.007066
Gold,-0.000357,0.002426,0.002175,0.003875,0.001124,0.000787,0.000774,0.002801,0.019020,0.006836
Commodities,0.005273,0.008212,0.005775,0.010617,-0.002675,-0.001119,0.005875,0.007066,0.006836,0.026368


In [18]:
covmat = covmat.values.copy()
covmat

array([[ 0.02325209,  0.01998246,  0.01730011,  0.02282169, -0.00406014,
        -0.00202108,  0.01918081,  0.0122861 , -0.00035668,  0.00527258],
       [ 0.01998246,  0.03127645,  0.01993007,  0.02801568, -0.00525241,
        -0.0024292 ,  0.02087001,  0.01721907,  0.00242641,  0.00821205],
       [ 0.01730011,  0.01993007,  0.03702766,  0.02445774, -0.00381302,
        -0.00188129,  0.01582384,  0.01394245,  0.00217481,  0.00577538],
       [ 0.02282169,  0.02801568,  0.02445774,  0.04606858, -0.00589978,
        -0.00294834,  0.02646184,  0.02105786,  0.00387464,  0.01061698],
       [-0.00406014, -0.00525241, -0.00381302, -0.00589978,  0.01010761,
         0.00473312, -0.0036533 , -0.00357252,  0.00112417, -0.00267518],
       [-0.00202108, -0.0024292 , -0.00188129, -0.00294834,  0.00473312,
         0.00277323, -0.00179335, -0.00162941,  0.00078697, -0.00111948],
       [ 0.01918081,  0.02087001,  0.01582384,  0.02646184, -0.0036533 ,
        -0.00179335,  0.0389648 ,  0.01640805

In [19]:
def MinVol_objective(x) :
        variance = x.dot(covmat).dot(x.T)  ## 여기서 error 
        sigma = variance ** 0.5
        return (sigma)

In [20]:
def weight_sum_constraint(x) :
        return(x.sum() - 1.0 )

In [21]:
def MinVol(covmat, lb, ub) :
    x0 = np.repeat(1/covmat.shape[1], covmat.shape[1])  ##1/covmat 이게 무슨의미?
    lbound  = np.repeat(lb, covmat.shape[1])
    ubound  = np.repeat(ub, covmat.shape[1])
    bnds = tuple(zip(lbound, ubound))
    
    constraints = ({'type': 'eq', 'fun': weight_sum_constraint})
    options = {'ftol': 1e-20, 'maxiter': 800}
    
    result = minimize(fun = MinVol_objective,
                      x0 = x0,
                      method = 'SLSQP',
                      constraints = constraints,
                      options = options,
                      bounds = bnds)
    return(result.x)

In [26]:
x0 = np.repeat(1/covmat.shape[1], covmat.shape[1]) #가중치 모두 동일하게 equal weight!
x0

array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])

In [28]:
lbound  = np.repeat(0.05, covmat.shape[1])
ubound  = np.repeat(0.2, covmat.shape[1])
bnds = tuple(zip(lbound, ubound))
bnds #가중치 최대 최소 제한

((0.05, 0.2),
 (0.05, 0.2),
 (0.05, 0.2),
 (0.05, 0.2),
 (0.05, 0.2),
 (0.05, 0.2),
 (0.05, 0.2),
 (0.05, 0.2),
 (0.05, 0.2),
 (0.05, 0.2))

In [29]:
constraints = ({'type': 'eq', 'fun': weight_sum_constraint}) #가중치 합이 1
options = {'ftol': 1e-20, 'maxiter': 800}

In [ ]:
result = minimize(fun = MinVol_objective,
                      x0 = x0,
                      method = 'SLSQP',
                      constraints = constraints,
                      options = options,
                      bounds = bnds)

In [171]:
print(result)
print('*'*100)
print('weight :',result.x)

     fun: 0.06368372536409718
     jac: array([0.08264039, 0.10734561, 0.10172827, 0.13701813, 0.02456549,
       0.0138894 , 0.10606487, 0.08921534, 0.08264039, 0.08264039])
 message: 'Positive directional derivative for linesearch'
    nfev: 144
     nit: 16
    njev: 12
  status: 8
 success: False
       x: array([0.06319558, 0.05      , 0.05      , 0.05      , 0.2       ,
       0.2       , 0.05      , 0.05      , 0.19204934, 0.09475508])
****************************************************************************************************
weight : [0.06319558 0.05       0.05       0.05       0.2        0.2
 0.05       0.05       0.19204934 0.09475508]
